In [1]:
from pprint import pprint
import argparse
import gc
from os.path import join
from datetime import datetime

import torch
from torch.utils.data import DataLoader
import lightning as L
from lightning.pytorch.loggers import WandbLogger
from data.dataset import CIFAKEDataset

# from coco_fake_dataset import COCOFakeDataset
# from dffd_dataset import DFFDDataset

from model import model
from lib.util import load_config
import random
import numpy as np



/home/vivek/anaconda3/envs/artifact-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def args_func():
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--cfg",
        type=str,
        help="The path to the config.",
        default="./configs/results_cifake_T.cfg",
    )
    args = parser.parse_args()
    return args


In [7]:
gc.collect()
torch.cuda.empty_cache()

In [3]:
print("hi")

hi


In [4]:
# args = args_func()
# load configs
# cfg = load_config(args.cfg)
cfg = load_config("./configs/results_cifake_T.cfg")
pprint(cfg)

{'dataset': {'cifake_path': '../../datasets/cifake',
             'coco2014_path': '../../datasets/coco2014',
             'coco_fake_path': '../../datasets/fake_coco',
             'dffd_path': '../../datasets/dffd',
             'labels': 2,
             'name': 'cifake'},
 'model': {'add_fft_channel': True,
           'add_lbp_channel': True,
           'add_magnitude_channel': False,
           'backbone': 'BNext-T',
           'freeze_backbone': True},
 'test': {'batch_size': 32,
          'limit_test_batches': 1.0,
          'mixed_precision': True,
          'resolution': 224,
          'seed': 5,
          'weights_path': './weights'},
 'train': {'accumulation_batches': 4,
           'batch_size': 32,
           'epoch_num': 10,
           'limit_train_batches': 1.0,
           'limit_val_batches': 1.0,
           'mixed_precision': True,
           'resolution': 224,
           'seed': 5}}


In [ ]:
# preliminary setup
torch.manual_seed(cfg["test"]["seed"])
random.seed(cfg["test"]["seed"])
np.random.seed(cfg["test"]["seed"])
torch.set_float32_matmul_precision("medium")

# get data
if cfg["dataset"]["name"] == "cifake":
    print(f"Loading CIFAKE dataset from {cfg['dataset']['cifake_path']}")
    test_dataset = CIFAKEDataset(
        dataset_path=cfg["dataset"]["cifake_path"],
        split="test",
        resolution=cfg["test"]["resolution"],
    )

In [5]:
# loads the dataloaders
num_workers = 4
test_loader = DataLoader(
    test_dataset,
    batch_size=cfg["test"]["batch_size"],
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True,
)

NameError: name 'test_dataset' is not defined

In [6]:
# init model
net = model.BNext4DFR.load_from_checkpoint(join(cfg["test"]["weights_path"], f"{cfg['dataset']['name']}_{cfg['model']['backbone'][-1]}{'_unfrozen' if not cfg['model']['freeze_backbone'] else ''}.ckpt"))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = net.to(device)

Model with ELM Attention and Infor-Recoupling


In [ ]:

    # # start training
    # date = datetime.now().strftime("%Y%m%d_%H%M")
    # project = "DFAD_CVPRW24"
    # run_label = args.cfg.split("/")[-1].split(".")[0]
    # run = cfg["dataset"]["name"] + f"_test_{date}_{run_label}"
    # logger = WandbLogger(project=project, name=run, id=run, log_model=False)
    # trainer = L.Trainer(
    #     accelerator="gpu" if "cuda" in str(device) else "cpu",
    #     devices=1,
    #     precision="16-mixed" if cfg["test"]["mixed_precision"] else 32,
    #     limit_test_batches=cfg["test"]["limit_test_batches"],
    #     logger=logger,
    # )
    # trainer.test(model=net, dataloaders=test_loader)